In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [2]:
from dataclasses import dataclass

@dataclass
class NNParams:
    batch_size: int = 64
    test_batch_size: int = 1000
    epochs: int = 14
    lr: float = 1.0
    gamma: float = 0.7
    dry_run: bool = False
    seed: int = 1.0
    log_interval: int = 10
        
default_params = NNParams()

In [3]:
args = default_params
device = torch.device("cuda")

train_kwargs = {'batch_size': args.batch_size}
test_kwargs = {'batch_size': args.test_batch_size}
cuda_kwargs = {'num_workers': 1,
               'pin_memory': True,
               'shuffle': True}
train_kwargs.update(cuda_kwargs)
test_kwargs.update(cuda_kwargs)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('../data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302544
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.233739
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.075457
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.997094
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.378395
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.491233
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.538211
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.208116
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.459793
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.117341
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.176622
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.307091
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.221081
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.291379
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.254495
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.216245
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.319315
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.296474
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.131533
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.232901
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.028899
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.138599
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.009327
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.001422
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.025906
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.023757
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.010926
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.020142
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.081856
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.022408
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.040950
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.010353
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.046258
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.080947
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.166601
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.068740
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.028954
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.086866
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.056352
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.011147


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.007728
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.001325
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.051344
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.080282
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.025791
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.185759
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.011950
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.039570
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.011901
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.030553
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.045714
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.042853
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.058522
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.077574
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.033224
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.015597
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.003272
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.051216
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.009347
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.008382


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.049993
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.179493
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.176760
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.006786
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.249523
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.030763
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.027007
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.009692
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.015199
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.031525
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.010190
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.002151
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.017339
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.025401
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.005827
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.027462
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.006499
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.019811
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.010995
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.014799


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.060088
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.011708
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.002018
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.332625

Test set: Average loss: 0.0253, Accuracy: 9921/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.018861
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.034490
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.172678
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.004455
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.004032
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.034708
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.006033
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.028375
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.007629
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.001918
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.002016
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.012836
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.001848
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.003650
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.076758
Train Epoch: 8 [9

Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.015386
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.017333
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.004149
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.004670
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.188394
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.050906
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.050539
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.013173
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.009572
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.001238
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.023425
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.043148
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.007418
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.008945
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.020922
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.011085
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.001469
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.020037
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.002438
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.026156


Train Epoch: 11 [24960/60000 (42%)]	Loss: 0.007266
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.001501
Train Epoch: 11 [26240/60000 (44%)]	Loss: 0.072091
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.008913
Train Epoch: 11 [27520/60000 (46%)]	Loss: 0.009207
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.013024
Train Epoch: 11 [28800/60000 (48%)]	Loss: 0.003421
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.057316
Train Epoch: 11 [30080/60000 (50%)]	Loss: 0.004088
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.020075
Train Epoch: 11 [31360/60000 (52%)]	Loss: 0.004814
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.019093
Train Epoch: 11 [32640/60000 (54%)]	Loss: 0.021569
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.002743
Train Epoch: 11 [33920/60000 (57%)]	Loss: 0.004431
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.002797
Train Epoch: 11 [35200/60000 (59%)]	Loss: 0.021376
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.001352
Train Epoch: 11 [36480/60000 (61%)]	Loss: 0.028966
Train Epoch: 11 [37120/60000 (6

Train Epoch: 13 [7040/60000 (12%)]	Loss: 0.055924
Train Epoch: 13 [7680/60000 (13%)]	Loss: 0.002119
Train Epoch: 13 [8320/60000 (14%)]	Loss: 0.033793
Train Epoch: 13 [8960/60000 (15%)]	Loss: 0.001948
Train Epoch: 13 [9600/60000 (16%)]	Loss: 0.002376
Train Epoch: 13 [10240/60000 (17%)]	Loss: 0.094434
Train Epoch: 13 [10880/60000 (18%)]	Loss: 0.037577
Train Epoch: 13 [11520/60000 (19%)]	Loss: 0.027986
Train Epoch: 13 [12160/60000 (20%)]	Loss: 0.001384
Train Epoch: 13 [12800/60000 (21%)]	Loss: 0.287136
Train Epoch: 13 [13440/60000 (22%)]	Loss: 0.005491
Train Epoch: 13 [14080/60000 (23%)]	Loss: 0.055382
Train Epoch: 13 [14720/60000 (25%)]	Loss: 0.021658
Train Epoch: 13 [15360/60000 (26%)]	Loss: 0.065825
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.013609
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.035649
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.020595
Train Epoch: 13 [17920/60000 (30%)]	Loss: 0.002589
Train Epoch: 13 [18560/60000 (31%)]	Loss: 0.001308
Train Epoch: 13 [19200/60000 (32%)]	

Train Epoch: 14 [50560/60000 (84%)]	Loss: 0.022047
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.010278
Train Epoch: 14 [51840/60000 (86%)]	Loss: 0.012887
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.004272
Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.003115
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.040859
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.005454
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.002013
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.025074
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.001295
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.024934
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.037378
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.027832
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.017632
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.006005

Test set: Average loss: 0.0253, Accuracy: 9919/10000 (99%)

